In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.metrics import log_loss
from scipy.stats import chi2_contingency, ttest_ind
import patsy
from lifelines import CoxPHFitter 


In [2]:
# Add the directory to sys.path
import sys
module_path = Path('./../code')
sys.path.append(str(module_path))
import utils

In [3]:
import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

# Displays all the columns, does 
pd.set_option('display.max_columns', None)

In [4]:
# Define directory path variable
data_dir = Path('./../data/')
notebook_dir = Path('./../notebook')
results_dir = Path('./../results/')

# Define data filenames
data_txt_file = "all_data_files.txt" 
ugi_file = "ugicancer_registry_clean.csv"

# Filenames to be saved as 
today_date = datetime.now().strftime('%Y%m%d') # Get today's date
consort_diagram_numbers_filename = "consort_diagram_numbers.txt"

# final_data_file = f'cleaned_cohort_{today_date}.csv'
# final_demtable_file = f'demtable_{today_date}.csv'
# final_demtablegrouped_file = f'demtable_grouped_{today_date}.csv'

### Apply exclusion criteria

Algorithm to apply exclusion criteria on the data batches: 
1. Iterate through all the batches and apply the exclusion criteria for screening age
2. Merge the batches together 
3. Drop duplicates, applying the exclusion criteria for multiple encounters
4. Merge with UGI cancer 
5. Add BMI variable
6. Apply UGI cancer time and BMI exclusion criteria 

In [5]:
file_list = ['data_2023-12-01_2023-12-02.csv.gz', 'data_2023-12-03_2023-12-12.csv.gz']

list_chunks = [] # To store the list of data chunks

# Keep track of number of encounters and unique patients 
num_enc_incl = 0
pts_incl = set()
num_enc_excl_age = 0

for filename in file_list:
    df_chunk = pd.read_csv(data_dir / filename, low_memory=False)[utils.COLNAMES_COHORT]
    print(f'{filename}: {df_chunk.shape}')

    # Update numbers 
    num_enc_incl += df_chunk.shape[0]
    pts_incl.update(df_chunk.pt_id)

    # Apply screening age exclusion criteria 
    excl_age = (df_chunk.age < 40) | (df_chunk.age > 85) # Pts not within proposed screening age of 40-85 
    num_enc_excl_age += int(excl_age.sum())
    list_chunks.append(df_chunk[~excl_age]) 

df_chunks = pd.concat(list_chunks, ignore_index=True)
num_pt_excl_age = len(pts_incl) - df_chunks.pt_id.nunique()

data_2023-12-01_2023-12-02.csv.gz: (28504, 163)
data_2023-12-03_2023-12-12.csv.gz: (205654, 163)


In [6]:
# Apply multiple encounters exclusion criteria 
df_patients = df_chunks.sort_values(by='visit_start_date').drop_duplicates(subset='pt_id', keep='first')
num_enc_excl_duplicates = df_chunks.shape[0] - df_patients.shape[0]
num_pt_excl_duplicates = df_chunks.pt_id.nunique() - df_patients.pt_id.nunique()

In [7]:
# Read UGI cancer registry data 
df_ugi = pd.read_csv(data_dir / ugi_file, low_memory=False) #, index_col = False)

# Select the first UGI cancer diagnosis by date for patients with multiple UGI cancers 
df_ugi_first = df_ugi.sort_values(by='datetime_dx').drop_duplicates(subset='mrn', keep='first')

# Merge UGI registry with extracted Epic data 
df_merged = df_patients.merge(df_ugi_first, on='mrn', how='left')

print(f'Number of patients in registry without matching MRN: {df_ugi_first.shape[0] - df_merged.datetime_dx.notnull().sum()}')
print(f'Number of patients in registry with matching MRN: {int(((df_merged.pt_id.notnull()) & df_merged.datetime_dx.notnull()).sum())}')

Number of patients in registry without matching MRN: 2537
Number of patients in registry with matching MRN: 165


In [8]:
# Convert height from inches to meters and weight from ounces to kg
inches_to_meters = 0.0254
ounces_to_kg = 0.0283495

df_merged['height_baseline_m'] = df_merged['height_baseline'] * inches_to_meters
df_merged['weight_baseline_kg'] = df_merged['weight_baseline'] * ounces_to_kg
df_merged['BMI_baseline_all'] = df_merged['weight_baseline_kg'] / df_merged['height_baseline_m']**2

# Count and calculate the percentage of non-null values
columns = ["height_baseline", "weight_baseline", "BMI_baseline", 'BMI_baseline_all']
non_null_counts = df_merged[columns].notnull().sum()
non_null_percentages = (non_null_counts / len(df_merged)) * 100

summary = pd.DataFrame({
    "non_null_count": non_null_counts,
    "non_null_percent": non_null_percentages.round(2)
})
print(summary)


                  non_null_count  non_null_percent
height_baseline            81703             88.76
weight_baseline            68324             74.23
BMI_baseline               66087             71.80
BMI_baseline_all           68017             73.89


In [9]:
# Convert to datetime 
date_vars = ['visit_start_date', 'datetime_dx', 'date_of_death']
for date_var in date_vars: 
    df_merged[date_var] = pd.to_datetime(df_merged[date_var], errors='coerce')

# Define exclusion criteria 
excl_dx_before_visit = (df_merged.visit_start_date > df_merged.datetime_dx) # Pts whose UGI cancer diagnosis occured before the visit date
excl_dx_soon_after_visit = ((df_merged.visit_start_date + pd.DateOffset(months=12)) >= df_merged.datetime_dx) # Pts whose UGI cancer diagnosis occured less than 12 months after the visit date
excl_gastrichx = (df_merged.gastricca == 1.0) & (df_merged.primary_tumor_site.isna()) # Pts who had a hx of gastric cancer as part of their PMHx but not included in the registry
excl_esophagealhx = (df_merged.esophagealca == 1.0) & (df_merged.primary_tumor_site.isna()) # Pts who had a hx of esophageal cancer as part of their PMHx but not included in the registry
excl_otherugicahx = (df_merged.ugica_other == 1) # Pts who were in the registry for other UGI cancer 
excl_death = (df_merged.visit_start_date <= df_merged.date_of_death) # Pts whose death date is documented as prior to or day of encounter date
excl_bmi_missing = (df_merged.BMI_baseline_all.isna()) # Pts who has missing BMI, aka not seen in person for the last 6 months 

# Calculate number of pts excluded
num_pt_excl_dx_before_visit = excl_dx_before_visit.sum()
num_pt_excl_dx_soon_after_visit = excl_dx_soon_after_visit.sum()
num_pt_excl_gastrichx = excl_gastrichx.sum()
num_pt_excl_esophagealhx = excl_esophagealhx.sum()
num_pt_excl_otherugicahx = excl_otherugicahx.sum()
num_pt_excl_death = excl_death.sum()
num_pt_excl_bmi_missing = excl_bmi_missing.sum()

# Apply exclusion criteria 
df_cohort = df_merged[~(excl_dx_before_visit | excl_dx_soon_after_visit | excl_gastrichx | excl_esophagealhx | excl_otherugicahx | excl_death | excl_bmi_missing)]
df_cohort.shape

(66134, 179)

In [10]:
# Total (non-sequential) exclusion numbers for cohort
output = (
    f'Total encounters: {num_enc_incl}, total patients: {len(pts_incl)}\n'
    f'Excluded - outside screening age: {num_enc_excl_age} encounters, {num_pt_excl_age} pts\n'
    f'Excluded - multiple encounters: {num_enc_excl_duplicates} encounters, {num_pt_excl_duplicates} pts\n'
    f'Excluded - UGI cancer dx before visit: {num_pt_excl_dx_before_visit} patients\n'
    f'Excluded - UGI cancer dx soon after visit: {num_pt_excl_dx_soon_after_visit} patients\n'
    f'Excluded - gastric ca hx not confirmed: {num_pt_excl_gastrichx} patients\n'
    f'Excluded - esophageal ca hx not confirmed: {num_pt_excl_esophagealhx} patients\n'
    f'Excluded - other UGI cancer subtype: {num_pt_excl_otherugicahx} patients\n'
    f'Excluded - death prior to enc: {num_pt_excl_death} patients\n'
    f'Excluded - BMI missing: {num_pt_excl_bmi_missing} patients\n'
    f'Cohort: {df_cohort.shape[0]} encounters, {df_cohort.pt_id.nunique()} patients\n'
)

print(output)

# Write the output string to a file
with open(data_dir / consort_diagram_numbers_filename, "w") as file:
    file.write(output)


Total encounters: 234158, total patients: 139640
Excluded - outside screening age: 73979 encounters, 47594 pts
Excluded - multiple encounters: 68133 encounters, 0 pts
Excluded - UGI cancer dx before visit: 165 patients
Excluded - UGI cancer dx soon after visit: 165 patients
Excluded - gastric ca hx not confirmed: 24 patients
Excluded - esophageal ca hx not confirmed: 547 patients
Excluded - other UGI cancer subtype: 119 patients
Excluded - death prior to enc: 1357 patients
Excluded - BMI missing: 24029 patients
Cohort: 66134 encounters, 66134 patients



### Clean data

In [11]:
def clean_data(df):
    df.visit_start_date = pd.to_datetime(df.visit_start_date)
    df.datetime_dx = pd.to_datetime(df.datetime_dx)
    df.date_of_death = pd.to_datetime(df.date_of_death, format='mixed')

    # Extract year from visit dates
    df['visit_year'] = df.visit_start_date.dt.year
    df['diagnosis_year'] = df.datetime_dx.dt.year
    df['death_year'] = df.date_of_death.dt.year

    # Create two variables per categorical where missing = a category and null for easier data processing later
    # _missing will have nulls
    df['sex_missing'] = np.where(df.sex == "No matching concept", np.nan, df.sex) 
    df['ethnicity_missing'] = np.where(df.ethnicity == "No matching concept", np.nan, df.ethnicity) 

    # Clean up race variable 
    df['race_clean'] = df.race.str.lower().map(utils.RACE_DICT)
    df['race_clean_missing'] = np.where(df.race_clean == "No matching concept", np.nan, df.race_clean)

    # Create two cleaned H pylori variables
    # Impute missing = 0 
    df['hpylori_active'] = df.apply(utils.clean_hpylori, axis=1) # Only stool and breath testing
    df['hpylori_active_chronic_missing'] = df.apply(utils.clean_hpylori_serology, axis=1) # Incorporate Hpylori serology and PMHx
    df['hpylori_active_chronic_binary'] = df['hpylori_active_chronic_missing'].apply(lambda x: 1 if x==1 else 0)
    df['hpylori_active_chronic'] = df['hpylori_active_chronic_missing'].apply(lambda x: x if x in [0, 1] else -1)

    # Create a comprehensive tobacco and alcohol variable by merging data obtained from PMHx and social history.
    df['tobacco_all'] = df[['tobacco', 'social_smoking_ever']].max(axis=1)
    df['alcohol_all'] = df[['alcohol', 'social_alcohol']].max(axis=1)

    # Create additional vars for alcohol and tobacco to analyze 
    df['alcohol_all_missing'] = df['alcohol_all'].apply(lambda x: np.nan if x == -1 else x)
    df['alcohol_binary_missing'] = df['alcohol_all_missing'].apply(lambda x: 1 if x == 2 else x)
    df['alcohol_binary'] = df['alcohol_all'].apply(lambda x: 1 if x in [1,2] else 0)

    df['tobacco_all_missing'] = df['tobacco_all'].apply(lambda x: np.nan if x == -1 else x)
    df['tobacco_binary_missing'] = df['tobacco_all_missing'].apply(lambda x: 1 if x == 2 else x)
    df['tobacco_binary'] = df['tobacco_all'].apply(lambda x: 1 if x in [1,2] else 0)

    # Make PMHx and FMHx binary 
    df['hnca'] = df['hnca'].apply(lambda x: 1 if x == 1 else 0)
    df['achalasia'] = df['achalasia'].apply(lambda x: 1 if x == 1 else 0)
    df['pud'] = df['pud'].apply(lambda x: 1 if x == 1 else 0)
    df['gerd'] = df['gerd'].apply(lambda x: 1 if x == 1 else 0)
    df['barretts'] = df['barretts'].apply(lambda x: 1 if x == 1 else 0)
    df['cad'] = df['cad'].apply(lambda x: 1 if x == 1 else 0)
    df['famhx_cancer'] = df['famhx_cancer'].apply(lambda x: 1 if x == 1 else 0)
    df['famhx_gastricca'] = df['famhx_gastricca'].apply(lambda x: 1 if x == 1 else 0)
    df['famhx_esophagealca'] = df['famhx_esophagealca'].apply(lambda x: 1 if x == 1 else 0)
    df['famhx_colonca'] = df['famhx_colonca'].apply(lambda x: 1 if x == 1 else 0)
    df['famhx_barretts'] = df['famhx_barretts'].apply(lambda x: 1 if x == 1 else 0)

    # Categorize medication use date
    df['PPI'] = df['PPI_start_date'].notna().astype(int)
    df['ASA'] = df['ASA_start_date'].notna().astype(int)
    df['NSAID'] = df['NSAID_start_date'].notna().astype(int)

    # Impute hemoglobin 
    hgball_mean = df.hgb_baseline.mean()
    df['hgball_baseline_imputed_mean'] = np.where(df.hgball_baseline.isna(), hgball_mean, df.hgball_baseline)

    # Create column for all UGI cancers together (stomach and esophagus)
    df['ugica'] = df[['ugica_ESCC', 'ugica_EAC', 'ugica_CGC', 'ugica_NCGC']].max(axis=1)
    df.loc[df.ugica.isna(), 'ugica'] = 0

    # Create other outcome variables 
    df['death'] = df.death_year.notna().astype(int)
    
    # Calculate the days between visit_start_date and datetime_dx, and visit_start_date and date_of_death
    df['days_to_dx'] = (df['datetime_dx'] - df['visit_start_date']).dt.days
    df['days_to_death'] = (df['date_of_death'] - df['visit_start_date']).dt.days

    # Create the days_to_event column as the minimum of days_to_dx and days_to_death
    df['days_to_event'] = df[['days_to_dx', 'days_to_death']].min(axis=1)
    df['months_to_event'] = df['days_to_event'] / 30.4375

    return df 

df_all = clean_data(df_cohort)

/var/folders/2m/nz3ptmfs6c586nfvk00y2dsc0000gp/T/ipykernel_47370/375052250.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.visit_start_date = pd.to_datetime(df.visit_start_date)
/var/folders/2m/nz3ptmfs6c586nfvk00y2dsc0000gp/T/ipykernel_47370/375052250.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.datetime_dx = pd.to_datetime(df.datetime_dx)
/var/folders/2m/nz3ptmfs6c586nfvk00y2dsc0000gp/T/ipykernel_47370/375052250.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [12]:
df = df_all[utils.VARS_TO_ANALYZE]
df.shape

(66134, 73)

In [13]:
df.head()

,ugica,death,subtype,visit_year,diagnosis_year,encounter_type,sex,sex_missing,race_clean,race_clean_missing,ethnicity,ethnicity_missing,social_language,alcohol_all,alcohol_all_missing,alcohol_binary,alcohol_binary_missing,tobacco_all,tobacco_all_missing,tobacco_binary,tobacco_binary_missing,hpylori_active,hpylori_active_chronic,hpylori_active_chronic_missing,hpylori_active_chronic_binary,hnca,achalasia,pud,gerd,cad,barretts,famhx_cancer,famhx_esophagealca,famhx_gastricca,famhx_colonca,famhx_barretts,ASA,PPI,NSAID,days_to_event,months_to_event,days_to_dx,days_to_death,height_baseline_m,weight_baseline_kg,BMI_baseline_all,BMI_baseline,hgball_baseline,hgball_baseline_imputed_mean,hgb_baseline,mcv_baseline,wbc_baseline,plt_baseline,sodium_baseline,potassium_baseline,chloride_baseline,bicarbonate_baseline,bun_baseline,scr_baseline,magnesium_baseline,calcium_baseline,phosphate_baseline,ast_baseline,alt_baseline,alp_baseline,tbili_baseline,tprotein_baseline,albumin_baseline,tsh_baseline,vitD_baseline,triglycerides_baseline,LDL_baseline,hgba1c_baseline
0,0.0,0,NaN,2023,NaN,Outpatient Visit,MALE,MALE,White,White,Not Hispanic or Latino,Not Hispanic or Latino,English,2.0,2.0,1,1.0,0.0,0.0,0,0.0,NaN,-1.0,NaN,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.803400,88.904032,27.336149,27.4,NaN,12.995855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0,NaN,2023,NaN,Outpatient Visit,FEMALE,FEMALE,No matching concept,NaN,Hispanic or Latino,Hispanic or Latino,Spanish,0.0,0.0,0,0.0,0.0,0.0,0,0.0,NaN,-1.0,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,1.440930,59.999449,28.897563,26.7,14.4,14.400000,14.4,92.7,10.1,361.0,140.0,3.1,108.0,NaN,6.0,0.50,NaN,8.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0,NaN,2023,NaN,Outpatient Visit,MALE,MALE,No matching concept,NaN,Hispanic or Latino,Hispanic or Latino,English,0.0,0.0,0,0.0,0.0,0.0,0,0.0,NaN,-1.0,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.672251,83.914520,30.007791,29.9,7.4,7.400000,7.4,86.6,7.3,166.0,142.0,4.3,108.0,22.0,69.0,7.03,2.0,7.3,5.9,20.0,15.0,101.0,0.2,6.5,3.4,NaN,32.2,NaN,NaN,6.2
4,0.0,0,NaN,2023,NaN,ED Visit,MALE,MALE,No matching concept,NaN,Hispanic or Latino,Hispanic or Latino,English,2.0,2.0,1,1.0,1.0,1.0,1,1.0,NaN,-1.0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,1.803400,72.574720,22.315223,23.7,NaN,12.995855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,0,NaN,2023,NaN,Outpatient Visit,FEMALE,FEMALE,No matching concept,NaN,Hispanic or Latino,Hispanic or Latino,English,2.0,2.0,1,1.0,1.0,1.0,1,1.0,NaN,-1.0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,1.526721,63.502880,27.244186,27.4,NaN,12.995855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
f'Incidence of UGI Cancer for our cohort: {df.ugica.value_counts()[1]/ df.ugica.value_counts()[0]*100:.2}%'

'Incidence of UGI Cancer for our cohort: 0.0015%'